In [2]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup
import time
import numpy as np
import json
from tqdm.notebook import tqdm

In [3]:
class Header:
    def __init__(self, header_text):
        self.name = 'h'
        self.header_text = header_text
        self.objects = []
    def __str__(self):
        result = f'**{self.header_text}**\n'
        for i in self.objects:
            result += f'{str(i)}\n'
        return result    
    def str_with_out_atrifacts(self):
        result = f'{self.header_text}\n'
        for i in self.objects:
            result += f'{i.str_with_out_atrifacts()}\n'
        return result 
    def append(self, o):
        self.objects.append(o)
    def get_pairs(self):
        result = []
        for i in self.objects:
            if i.name == 'h':
                result += i.get_pairs()
            else:
                result.append((self.header_text, str(i)))
        if len(result) > 0 and len(result[-1]) != 2:
            print(i, result[-1])
        return result

In [4]:
class Image:
    def __init__(self, tag):
        self.alt = tag.get('alt')
        self.src = tag.get('src')

In [5]:
class Paragraph:
    def __init__(self, tag, not_recursive_paragraph=True):
        self.name = tag.name
        self.paragraph_text = tag.text
        self.paragraph_text_with_artifacts = tag.text
        self.images = []
        for i in tag.find_all(True, recurcive=False):
            if i.name == 'strong' and not_recursive_paragraph and i.text != '':
                self.paragraph_text_with_artifacts = self.paragraph_text_with_artifacts.replace(
                    i.text, f' **{i.text}** ')
            elif i.name == 'a' and not_recursive_paragraph and i.text != '':
                self.paragraph_text_with_artifacts = self.paragraph_text_with_artifacts.replace(
                    i.text, f'[{i.text}]({i.get("href")})')
            elif i.name == 'em' and not_recursive_paragraph and i.text != '':
                self.paragraph_text_with_artifacts = self.paragraph_text_with_artifacts.replace(
                    i.text, f' __{i.text}__ ')
            elif i.name == 'img':
                self.images.append(Image(i))
            elif i.name in ('p', 'div', 'li', 'ul'):
                recurcive = Paragraph(i, False)
                self.paragraph_text_with_artifacts = self.paragraph_text_with_artifacts.replace(
                    recurcive.paragraph_text_with_artifacts, recurcive.paragraph_text)
                self.paragraph_text_with_artifacts = self.paragraph_text_with_artifacts.replace(
                    recurcive.paragraph_text, recurcive.paragraph_text_with_artifacts)
        self.paragraph_text_with_artifacts = self.paragraph_text_with_artifacts.replace('  ', ' ')
        self.paragraph_text_with_artifacts = self.paragraph_text_with_artifacts.replace('  ', ' ')
        self.paragraph_text_with_artifacts = self.paragraph_text_with_artifacts.replace('  ', ' ')
        self.paragraph_text_with_artifacts = self.paragraph_text_with_artifacts.replace('  ', ' ')
        self.paragraph_text_with_artifacts = self.paragraph_text_with_artifacts.replace('** **', '')
        self.paragraph_text_with_artifacts = self.paragraph_text_with_artifacts.replace('** **', '')
        self.paragraph_text_with_artifacts = self.paragraph_text_with_artifacts.replace('** **', '')
        self.paragraph_text_with_artifacts = self.paragraph_text_with_artifacts.replace('** **', '')
    def __str__(self):
        return self.paragraph_text_with_artifacts
    
    def str_with_out_atrifacts(self):
        return self.paragraph_text


In [6]:
def parse(soup, text):
    current_h1 = Header(text)
    current_h2 = None
    current_h3 = None
    current_h4 = None
    for tag in soup.find_all(True, recursive=False):
        if current_h4:
            current_header = current_h4
        elif current_h3:
            current_header = current_h3
        elif current_h2:
            current_header = current_h2
        else:
            current_header = current_h1
        if tag.name == 'h2':
            current_h2 = Header(tag.text)
            current_h1.append(current_h2)
            current_h3 = None
            current_h4 = None
        elif tag.name == 'h3':
            current_h3 = Header(tag.text)
            if current_h2:
                current_h2.append(current_h3)
            else:
                current_h1.append(current_h3)
            current_h4 = None
        elif tag.name == 'h4':
            current_h4 = Header(tag.text)
            if current_h3:
                current_h3.append(current_h4)
            elif current_h2:
                current_h2.append(current_h4)
            else:
                current_h1.append(current_h4)

        elif tag.name in ('p', 'div', 'ul', 'ol'):
            current_header.append(Paragraph(tag))
    return current_h1

In [7]:
r = rq.get('https://portal.hse.ru/personalpages')
web = BeautifulSoup(r.text, 'html.parser')
soup = web.find('div', {"class": "post__text"})
personalpages = parse(soup, 'Персональные страницы')

In [8]:
r = rq.get('https://portal.hse.ru/helpsite')
web = BeautifulSoup(r.text, 'html.parser')
soup = web.find('div', {"class": "post__text"})
helpsite = parse(soup, 'Как осуществляется доступ к редактированию сайта на портале?')

In [9]:
r = rq.get('https://portal.hse.ru/progs')
web = BeautifulSoup(r.text, 'html.parser')
soup = web.find('div', {'class': 'post__text'}).find('div', {"class": "with-indent5 _builder builder--text"})
progs = parse(soup, 'Инструкция по редактированию нового сайта образовательной программы')

In [10]:
r = rq.get('https://portal.hse.ru/poll')
web = BeautifulSoup(r.text, 'html.parser')
soup = web.find('div', {'class': 'post__text'}).find_all('div', {"class": "with-indent5 _builder builder--text"})[1:]
poll = Header('Регистрационная форма / опрос')
for i in soup:
    temp = parse(i, 'Регистрационная форма / опрос')
    poll.objects += temp.objects

In [11]:
r = rq.get('https://portal.hse.ru/im')
web = BeautifulSoup(r.text, 'html.parser')
soup = web.find('div', {'class': 'post__text'}).find_all('div', {"class": "with-indent5 _builder builder--text"})
im = Header('Создание и редактирование сайта подразделения')
for i in soup:
    temp = parse(i, 'Создание и редактирование сайта подразделения')
    im.objects += temp.objects

In [12]:
headers = [personalpages, helpsite, progs, poll, im]
for i in headers:
    for name, value in i.get_pairs():
        print(name)
        print(value)
        print()
    print()

У кого есть персональная страница и как она появляется на портале?
Сотрудники НИУ ВШЭ (за исключением сотрудников категории административно-хозяйственного персонала, не занимающих руководящие должности) обязаны иметь заполненную в соответствии со [Стандартом](https://www.hse.ru/docs/751535174.html) персональную страницу.

У кого есть персональная страница и как она появляется на портале?
Персональная страница появляется автоматически после внесения приказа о трудоустройстве сотрудника в базу [Управления персонала](http://hr.hse.ru/) (ЗиК, ИС-ПРО) и показывается в [общем списке преподавателей и сотрудников](http://www.hse.ru/org/persons/), а также в списках сотрудников на страницах подразделений.

У кого есть персональная страница и как она появляется на портале?
Автоматически появляются только страницы **штатных сотрудников и преподавателей, имеющих договор ГПХ ППС** (такие сотрудники показываются на портале с должностью «Приглашенный преподаватель»).

У кого есть персональная страница

In [13]:
headers